In [1]:
# Assignment 2 - 4)  Naive base with Bernoullifeature

import urllib2

import random
import math
import numpy as np
from math import log
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix


def separateDataByClass(total_data):
    
    separate_data = {}    
    for i in range(len(total_data)):
        mat_vector = total_data[i]
        
        if (mat_vector[-1] not in separate_data):
            separate_data[mat_vector[-1]] = []
        
        separate_data[mat_vector[-1]].append(mat_vector)
    
    print "Length of Train Data = ", len(separate_data[0])
    return separate_data

def membership_function(test_set, alpha, priorVal):
    value_1 = []
    value_2 = []
    
    for idx in alpha:
        if idx == 0.0 or idx == 1.0:
            value_1.append(0.0)
            value_2.append(0.0)
        else:
            value_1.append(log(idx))
            value_2.append(log(1 - idx))
            
    gX = [(np.sum(((test_set[:,j][i]*value_1[j]) + ((1-test_set[:,j][i])*value_2[j])) for j in range(test_set.shape[1]-1)) 
          + math.log(priorVal)) for i in range(test_set.shape[0])]
    return gX

def discriminant_function(g1X, g2X):
    
    predicted = []
    diff = [g2X[i] - g1X[i] for i in range(len(g1X))]
    
    for i in diff:
        if i > 0:
            predicted.append(1.0)
        else:
            predicted.append(0.0)
    
    return predicted

def confusion_matrixx(y_test_data, predicted_y, total_class):

    true_positive=0
    true_negative=0
    false_positive=0
    false_negative = 0
    
    conf_matrix = np.zeros((len(total_class),len(total_class)))
    
    for i in range(0, len(y_test_data)):
        if(y_test_data[i]==total_class[0] and (predicted_y[i]==total_class[0])):
            true_positive = true_positive + 1
        elif(y_test_data[i]==total_class[1] and (predicted_y[i]==total_class[1])):
            true_negative = true_negative + 1    
        elif(y_test_data[i]==total_class[0] and (predicted_y[i]==total_class[1])):
            false_positive  = false_positive + 1
        elif(y_test_data[i]==total_class[1] and (predicted_y[i]==total_class[0])):
            false_negative = false_negative + 1 
            
    conf_matrix[0,0]= true_positive
    conf_matrix[0,1]= false_positive
    conf_matrix[1,0]= false_negative
    conf_matrix[1,1]= true_negative
    
    return conf_matrix


def findOtherParameters(confusion_mat):
    
    list_diagonal = np.zeros(confusion_mat.shape[0])
    list_row_sum = np.zeros(confusion_mat.shape[0])
    list_column_sum=np.zeros(confusion_mat.shape[1])
    
    precision_value = []
    recall_value = []
    f_measure_value = []
    
    total = np.sum(confusion_mat)
    confuse_diagonal = 0

    for i in range(confusion_mat.shape[0]):
        for j in range(confusion_mat.shape[1]):
            list_row_sum[i] += confusion_mat[i][j]
            list_column_sum[i] += confusion_mat[j][i]
            if(i==j):
                list_diagonal[i] = confusion_mat[i][j]
                confuse_diagonal +=  confusion_mat[i][j]
    print "Accuracy", float(confuse_diagonal)/total
    accuracy = float(confuse_diagonal)/total

    for index in range(len(list_row_sum)):
        if list_row_sum[index]==0:
            precision_value.append(0.0)
        else:
            precision_value.append((float)(list_diagonal[index]) / list_row_sum[index])
            
        if list_column_sum[index]==0:
            recall_value.append(0)
        else:
            recall_value.append((float)(list_diagonal[index]) / list_column_sum[index])
        
        if precision_value[index]==0 or recall_value[index]==0:
            f_measure_value.append(0)
        else:
            f_measure_value.append((float) (2 * precision_value[index] * recall_value[index]) / (precision_value[index] + recall_value[index]))
        
    return accuracy, precision_value, recall_value, f_measure_value

def main():
    
    i=0
    temp = 0
    final_precision=0
    final_recall = 0
    final_fmeasure = 0
    final_accuracy = 0
    
    data = urllib2.urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data")
    inp_data = []
    target = []
    files = np.loadtxt(data,dtype=float, delimiter=',')
    
    files[files != 0] = 1
    print "\nK-Fold started "        
    kf = KFold(len(files), n_folds=10)
    print "No of folds = ",len(kf)
    
    for train_index, test_index in kf:
        
        trainingSet, testSet = files[train_index], files[test_index]
        
        trainingSet = np.array(trainingSet,dtype = 'float')
        testSet = np.array(testSet,dtype = 'float')
        
        # seperate data by class
        separated_data = separateDataByClass(trainingSet)
        
        class_1 = np.array(separated_data[0])
        class_2 = np.array(separated_data[1])
    
        c1_length = len(class_1)
        c2_length = len(class_2)
        
        # find prior array
        prior_array = [(c1_length*1.0/(c1_length+c2_length)), (c2_length*1.0/(c1_length+c2_length))]
        print "prior_array"
        print prior_array
    
        # find alph for seperate class
        alpha_1  = [(np.sum(class_1[:,i])/c1_length) for i in range(class_1.shape[1])]
        alpha_2  = [(np.sum(class_2[:,i])/c2_length) for i in range(class_2.shape[1])]
        
        # find membership function
        g1X = membership_function(testSet,alpha_1, prior_array[0])
        g2X = membership_function(testSet,alpha_2, prior_array[1])
        
        # find discriminanet function
        disc_function = discriminant_function(g1X, g2X)
        
        Y_Test_Data = []
#        no_of_class = []
        for i in  testSet[:,testSet.shape[1]-1]:
            Y_Test_Data.append(i)

        
        # find confusion matrix
        no_of_class = [0, 1]
        #confusion_mat = confusion_matrix(Y_Test_Data, disc_function)
        confusion_mat = confusion_matrixx(Y_Test_Data,disc_function,no_of_class)
        
        print confusion_mat
        
        # find precision, recall and f_measure
        accuracy, precision_val, recall_val, f_measure_val = findOtherParameters(confusion_mat)
     
        """print "Precision = ", precision_val
        print "Recall = ", recall_val
        print "F-Measure", f_measure_val
        print "\n Fold Completed \n"
        """
        if accuracy>temp:
            final_accuracy = accuracy
            final_precision = precision_val
            final_recall = recall_val
            final_fmeasure = f_measure_val
            temp = accuracy

        print "\n K-fold completed \n"
    
    # print errors
    print "Final Output : "
    print "Accuracy = ", final_accuracy
    print "Precision = ", final_precision
    print "Recall = ", final_recall
    print "F-Measure", final_fmeasure
    print "\n Fold Completed \n"
    
main()


K-Fold started 
No of folds =  10
Length of Train Data =  2788
prior_array
[0.6734299516908212, 0.32657004830917874]
[[   0.    0.]
 [  85.  376.]]
Accuracy 0.815618221258

 K-fold completed 

Length of Train Data =  2788
prior_array
[0.6732673267326733, 0.32673267326732675]
[[   0.    0.]
 [  86.  374.]]
Accuracy 0.813043478261

 K-fold completed 

Length of Train Data =  2788
prior_array
[0.6732673267326733, 0.32673267326732675]
[[   0.    0.]
 [  68.  392.]]
Accuracy 0.852173913043

 K-fold completed 

Length of Train Data =  2760
prior_array
[0.6665056749577397, 0.33349432504226034]
[[  28.    0.]
 [ 110.  322.]]
Accuracy 0.760869565217

 K-fold completed 

Length of Train Data =  2328
prior_array
[0.56218304757305, 0.43781695242695]
[[ 442.   18.]
 [   0.    0.]]
Accuracy 0.960869565217

 K-fold completed 

Length of Train Data =  2328
prior_array
[0.56218304757305, 0.43781695242695]
[[ 440.   20.]
 [   0.    0.]]
Accuracy 0.95652173913

 K-fold completed 

Length of Train Data =